In [ ]:
import sys

# This for managing relative imports from nb
if '..' not in sys.path: sys.path.append('..')

import pandas as pd
import numpy as np
import scipy.signal
from scipy.io import wavfile

import matplotlib
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore") # To supress WavFileWarning: Chunk (non-data) not understood, skipping it.

In [ ]:
#################################################
# Read the wav audio file into a stereo data file

rate, data = wavfile.read('../data/audio/88_Key_Ascending_Chromatic_Scale.wav')
#rate, data = wavfile.read('data/audio/88_Key_Descending_Chromatic_Scale.wav')
stereo_diff = pd.Series(data[:,0] - data[:,1])
data = data[:,0]

# Set up a wider notebook plot display size
# and plot a handful of views of the wav data
matplotlib.rcParams['figure.figsize'] = (20, 4)
print(matplotlib.rcParams['figure.figsize'])

series_data = pd.Series(data)
NFFT = int((rate / 1000) * 128) # 128ms window
moving_avg = series_data.rolling(NFFT).mean()
magnitudes = np.sqrt(data.astype(np.int32) * data)
envelope = np.abs(scipy.signal.hilbert(data))

plt.figure(1)
plt.title("Raw Time-Domain Wav Data")
plt.plot(data)

plt.figure(2)
plt.title("Magnitudes of Data (Zoomed In)")
plt.plot(magnitudes[0:125000])

plt.figure(3)
plt.title("Hilbert Envelope of Data (Zoomed In)")
plt.plot(envelope[0:125000])

plt.figure(4)
plt.title("Selected Audio Pulse (Zoomed In)")
plt.plot(series_data[34000:60000])

plt.figure(5)
plt.title("Stero Difference of Same Pulse")
plt.plot(stereo_diff[34000:60000])

plt.figure(6)
plt.title("128ms-Moving Average of Same Pulse")
plt.plot(moving_avg[34000:60000])

fft = np.fft.rfft(data)
window = 0.5 * (1 - np.cos(np.linspace(0, 2*np.pi, fft.shape[0], False)))

plt.figure(7)
plt.title("FFT Spectrum Windowed Across Full Wav File")
plt.plot(fft)

plt.figure(8)
plt.title("Plot of 2\u03C0 (Full Wav File) Hann Window Fcn")
plt.plot(window)

plt.figure(9)
plt.title("Full FFT Spectrum w/Hann Window Applied")
plt.plot(fft * window)

In [ ]:
################################################
# This cell is FFT params and some helper fcns

NOTE_NAMES = 'C C# D D# E F F# G G# A A# B'.split()

# These functions based on: https://newt.phys.unsw.edu.au/jw/notes.html
def freq_to_number(f): return int(round(69 + 12*np.log2(f/440.0)))
def number_to_freq(n): return 440 * 2.0**((n-69)/12.0)
def note_name(n): return NOTE_NAMES[n % 12] + str(int(n//12) - 1)

# At the low end, need frequency resolution of
# A#0 (29.135 Hz) - A0 (27.5 Hz) = 1.635 HZ
# (i.e. two lowest, half-step apart, piano keys)
NFFT = int(rate / 1.635)
FREQ_RESOLUTION = rate / NFFT
overlap = int(NFFT/2) 

# Only gonna look at FFT bins for piano key
# frequencies: A0 (27.5 Hz) to C8 (4186.0 Hz)
def freq_to_fftbin(freq): return freq/FREQ_RESOLUTION
def fftbin_to_freq(fft_bin): return fft_bin*FREQ_RESOLUTION
fft_imin = int(np.floor(freq_to_fftbin(27.5)))
fft_imax = int(np.ceil(freq_to_fftbin(4186.0)))

In [ ]:
def perform_fft(data, rate, NFFT, overlap):
#{
    # Perform the actual FFTs (using Hann window fcn)
    # Just clip the last segement that is < NFFT
    freq_array = np.fft.rfftfreq(NFFT, d=1.0/rate)
#     clipped_length = data.shape[0] - (data.shape[0] % NFFT)
#     hann_window_fcn = 0.5 * (1 - np.cos(np.linspace(0, 2*np.pi, NFFT, False)))
#     spectral = [np.fft.rfft(data[i:i+NFFT] * hann_window_fcn) for i in range(0, clipped_length, NFFT-overlap)]

    # Just clip the last segement that is < NFFT
    clipped_length = data.shape[0] - (data.shape[0] % NFFT)

    # Perform the actual FFTs (using Hann window fcn)
    hann_window_fcn = 0.5 * (1 - np.cos(np.linspace(0, 2*np.pi, NFFT, False)))
    
    # Power spectrum calculated as modulus/abs (which is real) of the complex numbers returned by FFT
    spectral = [np.abs(np.fft.rfft(data[i:i+NFFT] * hann_window_fcn)) for i in range(0, clipped_length, NFFT-overlap)]
    
    # Just collecting meta-data to print
    min_array, max_array = [], []
    min_piano_array, max_piano_array = [], []
    for spectrum in spectral:
    #{
        min_array.append(np.min(spectrum))
        max_array.append(np.max(spectrum))
        min_piano_array.append(np.min(spectrum[fft_imin:fft_imax]))
        max_piano_array.append(np.max(spectrum[fft_imin:fft_imax]))
    #}

    response_min = np.min(min_array)
    response_max = np.max(max_array)
    rsp_piano_min = np.min(min_piano_array)
    rsp_piano_max = np.max(max_piano_array)
    clipped_samps = data.shape[0] - clipped_length
    time_step_duration = (data.shape[0]*1000)/(rate*len(spectral))

    print(f"Wav file sampling rate: {rate} \
          \nWav file length: {data.shape[0]/rate} sec \
          \nClipping last: {clipped_samps} samps ({clipped_samps*1000/rate}ms) \
          \nNFFT: {NFFT} samples (~{int(round(NFFT*1000/rate))}ms) \
          \nOverlap: {overlap}  samples (~{int(round(overlap*1000/rate))}ms) \
          \nNumber of time steps: {len(spectral)} \
          \nDuration per time step: {time_step_duration}ms \
          \nSpectra piano freq window: \
          \n  min: 27.5 Hz (fft bin {fft_imin}) \
          \n  max: 4186.0 Hz (fft bin {fft_imax}) \
          \n  => x-axis plotting size = {fft_imax - fft_imin} \
          \nSpectra freq resolution: {FREQ_RESOLUTION} Hz \
          \nSpectra number frequencies: {NFFT//2+1} \
          \n  (check spectral[0].shape: {spectral[0].shape}) \
          \n  (check freq_array.shape: {freq_array.shape}) \
          \nSpectral response extremes: \
          \n  min (piano window):  {rsp_piano_min} \
          \n  min (full spectrum): {response_min} \
          \n  max (piano window):  {rsp_piano_max} \
          \n  max (full spectrum): {response_max} \
          \n  => y-axis plotting size: {rsp_piano_max-rsp_piano_min} \n")
    
    return spectral, response_min, response_max
#}

spectral, response_min, response_max = perform_fft(data, rate, NFFT, overlap)

plt.figure(1)
plt.title('Full FFT Spectrum at Time Step: ~31 Sec')
plt.plot(spectral[100])

plt.figure(2)
plt.title('Piano FFT Spectrum of Same Time Step (Effectively Zoomed In)')
plt.plot(spectral[100][fft_imin:fft_imax])

plt.figure(3)
plt.title('Piano Power Across Vs Time')
plt.plot([np.sum(spectrum[fft_imin:fft_imax]) for spectrum in spectral])

plt.figure(4)
plt.title('Mean of Power Spectrum Vs Time')
plt.plot([np.mean(spectrum[fft_imin:fft_imax]) for spectrum in spectral])

# plt.figure(5)
# plt.title('Peak FFT Rsp Across All Time Steps/Wav Data')
# plt.plot([np.max(spectrum) for spectrum in spectral])

In [ ]:
################################################
# Animation of FFT spectrum over the window of valid principal 
# piano frequencies (27.5 - 4186.0 Hz). Runs approx in real 
# time relative to the audio wav file.

from IPython.display import HTML
from matplotlib.animation import FuncAnimation

XPAD = 200
YPAD = 500000

def init():
#{
    # Set x-axis limits in Hz over piano freq window
    ax.set_xlim(fftbin_to_freq(fft_imin) - XPAD,
                fftbin_to_freq(fft_imax) + XPAD)

    # Set y-axis limits response extrema across entire animation
    ax.set_ylim(response_min - YPAD, response_max + YPAD)
    
    # Zero out data buffers
    xdata = np.zeros((fft_imax-fft_imin,))
    ydata = np.zeros((fft_imax-fft_imin,))
    ln.set_data(xdata, ydata)
    return ln,
#}

def run_animation(frame_number):
#{
    ydata = spectral[frame_number][fft_imin:fft_imax]
    xdata = np.linspace(fftbin_to_freq(fft_imin), 
                        fftbin_to_freq(fft_imax), 
                        fft_imax-fft_imin)
    
    ln.set_data(xdata, ydata)
    return ln,
#}

# Intialize plot and buffers
fig, ax = plt.subplots()
ax.set_xlabel('Hz'); ax.set_ylabel('Freq Response')
ln, = plt.plot([], [], animated=True)
xdata = np.zeros((fft_imax-fft_imin,))
ydata = np.zeros((fft_imax-fft_imin,))

# Run/render animation with approx 307.464ms time step duration
time_step_duration = (data.shape[0]*1000)/(rate*len(spectral))
ani = FuncAnimation(fig, run_animation, frames=np.arange(0, len(spectral)), 
                    init_func=init, interval=time_step_duration, blit=False, repeat=False)

# Display animation
HTML(ani.to_jshtml())

In [ ]:
# ani.save('88_Key_Ascending_Chromatic_Scale')
# ani.to_html5_video('88_Key_Ascending_Chromatic_Scale')